In [19]:
import os
import pandas as pd
import numpy as np
os.getcwd()

os.chdir('C:\\Users\\heram\\Documents\\Python Scripts\\ProjectWolf\\data\\input')

os.getcwd()

'C:\\Users\\heram\\Documents\\Python Scripts\\ProjectWolf\\data\\input'

In [20]:
#DF = pd.read_csv('USM00072249-drvd.txt', header=None ) 
#print(DF)

In [21]:
# Select Header Columns you need 
'''
recordID    record ID
ID          is the station identification code. See "igra2-stations.txt"
            for a complete list of stations and their names and locations.
YEAR        is the year of the sounding.
MONTH       is the month of the sounding.
DAY         is the day of the sounding.
HOUR        is the hour of the sounding (99 = missing).
RELTIME     is the release time of the sounding (format HHMM, missing=9999).
NUMLEV      is the number of levels in the sounding (i.e., the number of data records that follow).
PW          is the precipitable water (mm*100) between the surface and 500 hPa.
INVPRESS    is the pressure (in Pa or mb*100) at the level of the warmest temperature in the sounding. 
            Only provided if the warmest temperature is above the surface.
INVHGT      is the height (in meters above the surface) of the warmest temperature in the sounding.
            Only provided when the warmest temperature is above the surface.
INVTEMPDIF  is the difference between the warmest temperature in the sounding and the surface temperature (K * 10).
            Only provided if the warmest temperature is above the surface.
MIXPRESS    is the pressure (in Pa or mb * 100) at the top of the mixed layer as determined using the parcel method.
MIXHGT      is the height (in meters above the surface) of the top of the mixed layer 
            As determined using the parcel method.
FRZPRESS    is the pressure (in Pa or mb * 100) where the temperature first reaches the freezing point 
            when moving upward from the surface. Determined by interpolating linearly with respect to 
            the logarithm of pressure between adjacent reported levels.
            Not provided if the surface temperature is below freezing.
FRZHGT      is the height (in meters above the surface) where the temperature
            first reaches the freezing point when moving upward from the surface.
            Determined analogously to FRZPRESS. 
            Not provided if the surface temperature is below freezing.
LCLPRESS    is the pressure (in Pa or mb * 100) of the lifting condensation level.
LCLHGT      is the height (in meters above the surface) of the lifting condensation level.
LFCPRESS    is the pressure (in Pa or mb * 100) of the level of free convection.
LFCHGT      is the height (in meters above the surface) of the level of free convection.
LNBPRESS    is the pressure (in Pa or mb * 100) of the level of neutral buoyancy (or equilibrium level).
LNBHGT      is the height (in meters above the surface) of the level of neutral buoyancy (or equilibrium level).
LI          is the lifted index (in degrees C).
SI          is the Showalter index (in degrees C).
KI          is the K index (in degrees C).
TTI         is the total totals index (in degrees C).
CAPE        is the convective available potential energy (in J/kg).
CIN         is the convective inhibition (in J/kg).
'''
header_Columns = [
                    'recordID',
                    'ID',
                    'YEAR',
                    'MONTH',
                    'DAY',
                    'HOUR',
                    'RELTIME'
                    ]

In [22]:
# Select Data Columns you need 
'''
PRESS       is the reported pressure (Pa or mb * 100).
REPGPH      is the reported geopotential height (meters). This value is often not available at significant levels.
CALCGPH     is the calculated geopotential height (meters).
            The geopotential height has been estimated by applying the hydrostatic balance to the atmospheric layer 
            between the next lower level with a reported geopotential height and the current level.
TEMP        is the reported temperature (K * 10).
TEMPGRAD    is the temperature gradient between the current level and the next higher level with a temperature 
            [(K/km) * 10, positive if temperature increases with height].
PTEMP       is the potential temperature (K * 10).
PTEMPGRAD   is the potential temperature gradient between the current level and the next higher level 
            with a potential temperature [(K/km) * 10, positive if potential temperature increases with height].
VTEMP       is the virtual temperature (K * 10).
VPTEMP      is the virtual potential temperature (K * 10).
VAPPRESS    is the vapor pressure (mb * 1000) as computed from temperature, pressure, 
            and dewpoint depression at the same level.
SATVAP      is the saturation vapor pressure (mb * 1000) as computed from pressure and temperature at the same level.
REPRH       is the relative humidity (Percent * 10) as reported in the original sounding.
CALCRH      is the relative humidity (Percent * 10) as calculated from vapor pressure, saturation vapor pressure,
            and pressure at the same level.
RHGRAD      is the relative humidity gradient between the current level and the next higher usable level 
            [(%/km) * 10, positive if relative humidity increases with height].
UWND        is the zonal wind component [(m/s) * 10] as computed from the reported wind speed and direction.
UWDGRAD     is the vertical gradient of the zonal wind between the current level and the next higher level with 
            a wind observation [(m/s per km) * 10, positive if zonal wind becomes more positive with height].
VWND        is the meridional wind component [(m/s) * 10] as computed from the reported wind speed and direction.
VWNDGRAD    is the vertical gradient of the meridional wind component between the current level 
            and the next higher level with a wind observation [(m/s per km) * 10, positive if the meridional 
            wind becomes more positive with height].
N           is the refractive index (unitless).
'''
data_Columns = [ 
                'recordID',
                'PRESS',
                'REPGPH',
                'CALCGPH',
                'TEMP',
                'REPRH'
                ]

In [ ]:
def convertIGRA2(filename,header_Columns,data_Columns):
    
    #Import the file as panda data frame
    DF = pd.read_csv(filename, header=None ) #print (len(DF))
    
    #assign column name as data
    DF.columns = ['data']
    
    #Strip leading and trailing spaces in data
    DF['data'] = DF['data'].str.strip()
    
    #Create Header and record Indicators
    DF['Header'] =  np.where(DF['data'].str.contains('#'),1,0)
    DF['recordID'] = 0
    
    #Create Header and record number
    ifor_val = 0
    for i, row in DF.iterrows():
        if DF.at[i,'Header'] == 1:
            ifor_val = ifor_val + 1
        DF.at[i,'recordID'] = ifor_val
    
    # Create separate data frames for processing header and data records separately
    DF_Header = DF[DF['Header'] == 1]
    DF_data = DF[DF['Header'] == 0]
    
    #strip multiple spaces into a single space
    DF_Header = DF_Header.replace('\s+', ' ', regex=True)
    DF_data = DF_data.replace('\s+', ' ', regex=True)
    
    #replace -99999 to a missing value
    DF_Header = DF_Header.replace('-99999', ' ', regex=True)
    
    #replace # in headers
    DF_Header = DF_Header.replace('#', '', regex=True)
    
    #replacing spaces into a comma, 
    DF_Header = DF_Header.replace({' ': ','}, regex=True)
    DF_data = DF_data.replace({' ': ','}, regex=True)
    
    #Split data into 19 data columns as per IGRA2
    data_new = DF_data["data"].str.split(",", n = 19, expand = True)
    
    #Split header into 26 header records data columns as per IGRA2
    header_new = DF_Header["data"].str.split(",", n = 26, expand = True)
    
    #Header data finalized
    DF_header_final= pd.merge(DF_Header, header_new, left_index=True, right_index=True)
    del DF_header_final['data']
    del DF_header_final['Header']
    
    #Data records finalized
    DF_data_final= pd.merge(DF_data, data_new, left_index=True, right_index=True)
    del DF_data_final['data']
    del DF_data_final['Header']
    
    #Header columns identified and marked
    DF_header_final.columns = [
        'recordID',
        'ID',
        'YEAR',
        'MONTH',
        'DAY',
        'HOUR',
        'RELTIME',
        'NUMLEV',
        'PW',
        'INVPRESS',
        'INVHGT',
        'INVTEMPDIF',
        'MIXPRESS',
        'MIXHGT',
        'FRZPRESS',
        'FRZHGT',
        'LCLPRESS',
        'LCLHGT',
        'LFCPRESS',
        'LFCHGT',
        'LNBPRESS',
        'LNBHGT',
        'LI',
        'SI',
        'KI',
        'TTI',
        'CAPE',
        'CIN'
    ]
    
    #Replace 9999 value in RELTIME header column
    DF_header_final['RELTIME'] = DF_header_final['RELTIME'].replace('9999', np.NaN, regex=True)
    
    #subsetting the header columns
    #DF_header_final = DF_header_final[header_Columns]
    
    DF_header_final = DF_header_final.loc[:, header_Columns]
    
    #Data columns identified and marked
    DF_data_final.columns = [
        'recordID',
        'PRESS',
        'REPGPH',
        'CALCGPH',
        'TEMP',
        'TEMPGRAD',
        'PTEMP',
        'PTEMPGRAD',
        'VTEMP',
        'VPTEMP',
        'VAPPRESS',
        'SATVAP',
        'REPRH',
        'CALCRH',
        'RHGRAD',
        'UWND',
        'UWDGRAD',
        'VWND',
        'VWNDGRAD',
        'N'
    ]
    
    #subsetting the data columns
    #DF_data_final = DF_data_final[data_Columns]
    DF_data_final = DF_data_final.loc[:, data_Columns]
    
    #Merged the header and data records
    DF_final= pd.merge(DF_header_final, DF_data_final, on='recordID')
    
    #Replace -99999 as missing value 
    DF_final = DF_final.replace('-99999', np.NaN, regex=True)

    #write csv file
    DF_final.to_csv('DF_final.txt', header=True, index=False, sep=',')
    

    

In [ ]:
convertIGRA2('USM00072249-drvd.txt',header_Columns,data_Columns)